In [ ]:
%load_ext autoreload
%autoreload 2
import math
from pathlib import Path

import h5py
import numpy as np
from scipy import constants, signal, stats, optimize, integrate
from functools import partial
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cycler import cycler

import analysis
from basic.paths import (
    RESULTS_FOLDER,
    PARTICLE_VARIATION_FOLDER,
    DENSITY_VARIATION_FOLDER,
    THEORY_U_ALPHA_FILE,
    THEORY_DENSITY_RATIO_FILE,
    V_FLOW_VARIATION_FOLDER,
    THEORY_U_ALPHA_FILE,
    FOLDER_1D,
    FOLDER_2D,
    MPLSTYLE_FILE
)
from basic import (
    physics,
    Species,
    SpeciesInfo,
    RunInfo,
    Distribution
)

from plots import (
    settings,
    plots_1D,
    plots_2D,
    general,
)
from plots.settings import FIGURE_FULL_SIZE, FIGURE_HALF_SIZE

info = RunInfo(
    electron=SpeciesInfo(
        number_density=12.0e6,
        temperature=100.0,
        charge=-1,
        mass=1.0,
        bulk_velocity=0.0
    ),
    proton=SpeciesInfo(
        number_density=10.0e6,
        temperature=3.0,
        charge=+1,
        mass=1836.152674,
        bulk_velocity=0.0
    ),
    alpha=SpeciesInfo(
        number_density=1.0e6,
        temperature=12.0,
        charge=+2,
        mass=7294.29953,
        bulk_velocity=1.0e5
    )
)

save=False
plt.style.use(MPLSTYLE_FILE)
matplotlib.rcParams['figure.dpi'] = 100

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import qmc_quad

def indicatorRotatedRect(x, y, w, h, theta):
    x_rot = x * np.cos(theta) - y * np.sin(theta)
    y_rot = x * np.sin(theta) + y * np.cos(theta)
    return (np.abs(x_rot) <= w / 2) & (np.abs(y_rot) <= h / 2)

def generic_integrand(xy, w, h, theta, func):
    x, y = xy[0], xy[1]
    f1 = indicatorRotatedRect(x, y, w, h, +theta)
    f2 = indicatorRotatedRect(x, y, w, h, -theta)
    return func(x, y, (f1 | f2).astype(np.float64))

# Parameters
w, h = 2, 1.0
theta_vals = np.linspace(0, 60, 100) * np.pi / 180

tx = np.empty_like(theta_vals)
ty = np.empty_like(theta_vals)
n0 = np.empty_like(theta_vals)

# Compute T(θ) using qmc_quad
for i, theta in enumerate(theta_vals):
    f_n0 = lambda xy: generic_integrand(xy, w, h, theta, lambda x, y, f: f)
    f_tx = lambda xy: generic_integrand(xy, w, h, theta, lambda x, y, f: x ** 2 * f)
    f_ty = lambda xy: generic_integrand(xy, w, h, theta, lambda x, y, f: y ** 2 * f)

    n0_single, n0_err = qmc_quad(f_n0, a=[-w/2,-w/2], b=[w/2,w/2], n_estimates=2, n_points=10 ** 4)
    tx_single, tx_err = qmc_quad(f_tx, a=[-w/2,-w/2], b=[w/2,w/2], n_estimates=2, n_points=10 ** 4)
    ty_single, ty_err = qmc_quad(f_ty, a=[-w/2,-w/2], b=[w/2,w/2], n_estimates=2, n_points=10 ** 4)
    n0[i] = n0_single
    tx[i] = tx_single
    ty[i] = ty_single

In [ ]:
T_2D = (tx + ty) / (2 * n0)

plt.figure(figsize=(8, 5))
plt.plot(100 / np.cos(theta_vals), T_2D / np.max(T_2D) / (100 / np.cos(theta_vals)) ** 2, label='Temperature $T(\\theta)$')
plt.xlabel('Angle θ (degrees)')
plt.ylabel('Temperature T')
plt.title('Temperature vs. Rotation Angle θ')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# visualization of distribution overlap with interaction region
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def interactive_plot(u_alpha):
    width = 40
    c_s = info.c_s * 1e-3
    if u_alpha < c_s:
        theta = 0
    else:
        theta = np.arccos(c_s / u_alpha)

    t = np.linspace(0, c_s)
    s = np.linspace(-5 * c_s, 5 * c_s)
    plt.figure()
    # plt.text(0, 0, f"{u_alpha} {theta}")
    circle = plt.Circle((0,0), radius=c_s, fill=False, ls="--", lw=2)
    plt.plot(np.cos(theta) * t, np.sin(theta) * t, ls="solid")
    plt.plot(np.cos(theta) * t, -np.sin(theta) * t, ls="solid")
    plt.plot(c_s * np.cos(theta) + np.sin(theta) * s, c_s * np.sin(theta) - np.cos(theta) * s)
    plt.plot(c_s * np.cos(theta) + np.sin(theta) * s, -c_s * np.sin(theta) + np.cos(theta) * s)
    rect_pos = plt.Rectangle(
        xy=(
            (c_s - width / 2) * np.cos(theta) + np.sin(theta) * (-2 * c_s),
            (c_s - width / 2) * np.sin(theta) - np.cos(theta) * (-2 * c_s)
        ),
        width=1000, height=width, angle=theta * 180 / np.pi+270, alpha=0.4, edgecolor="black"
    )
    rect_neg = plt.Rectangle(
        xy=(
            +(c_s + width / 2) * np.cos(theta) + np.sin(theta) * (-2 * c_s),
            -(c_s + width / 2) * np.sin(theta) + np.cos(theta) * (-2 * c_s)
        ),
        width=1000, height=width, angle=-theta * 180 / np.pi+90, alpha=0.4, edgecolor="black"
    )
    plt.scatter(u_alpha, 0, s=40,zorder=5)
    plt.gca().add_patch(circle)
    plt.gca().add_patch(rect_pos)
    plt.gca().add_patch(rect_neg)
    plt.gca().set_aspect("equal")
    plt.xlim(-110, u_alpha+width)
    plt.ylim(-110, 110)
    plt.xlabel("Velocity $v_{\\alpha,x}$ (km/s)")
    plt.ylabel("Velocity $v_{\\alpha,y}$ (km/s)")
    plt.show()

interactive_plt = interactive(interactive_plot, u_alpha=(100.0, 200.0))
interactive_plt

In [ ]:
magnetic_field_strength = 4.346348316425957e-06# 45e-9
omega_c_alpha = magnetic_field_strength * info.alpha.si_charge / info.alpha.si_mass
omega_c_proton = magnetic_field_strength * info.proton.si_charge / info.proton.si_mass
print(f"B = {magnetic_field_strength * 1e4}")
print(info.omega_pp / omega_c_alpha)
print(info.omega_pp / omega_c_proton)
print(info.omega_pp / (10 * info.proton.si_charge / info.proton.si_mass))

# Simulation 2D

In [ ]:
# plot max-e-field energy
# TODO: Change markers & colors, perhaps change the format completely so that there is more space for the different markers (in that case focus on 100 and 140 km/s)
plots_2D.magneticFieldDirectionElectricField(info, save=False)
plt.show()

## Alpha flow-speed variation (B=0)

In [ ]:
plots_2D.maxEnergyVsAlphaFlowSpeed(info, normalize_energy=True, save=save)
plt.show()

In [ ]:
plots_2D.waveAngleVsAlphaFlowSpeed(info, "x", save=True)
plt.show()

In [ ]:
plots_2D.wavenumberVsAlphaFlowSpeed(info, save=True)
plots_2D.omegaVsAlphaFlowSpeed(info, save=True)
plt.show()

In [ ]:
files = sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5"))
flow_velocity = np.empty(len(files))
growth_rate = []
for file_idx, filename in enumerate(files):
    flow_velocity[file_idx] = int(filename.stem[-3:])
    with h5py.File(filename) as f:
        E_x = f['Electric Field/ex'][1:]
        E_y = f['Electric Field/ey'][1:]
        time = f["Header/time"][1:] * info.omega_pp
    res = analysis.fitGrowthRate(time, np.mean(E_x ** 2 + E_y ** 2, axis=(1,2)), allowed_slope_deviation=0.5)
    growth_rate.append(res[0].slope/2)
growth_rate = np.array(growth_rate)
print(np.max(np.abs(np.mean(growth_rate) - growth_rate)) / np.abs(np.mean(growth_rate)))
plt.plot(flow_velocity, growth_rate, ls="", marker="o", color="white", markeredgecolor="black", markeredgewidth=1)
with h5py.File(THEORY_U_ALPHA_FILE) as f:
        theory_v = f["u_alpha_bulk"][:] / 1e3
        theory_gamma = f["gamma_max"][:] / info.omega_pp
print(theory_gamma)
plt.plot(theory_v, theory_gamma)

In [ ]:

for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    for species in [Species.ALPHA]:
        with h5py.File(filename) as f:
            time = f["Header/time"][:]
            # print(f['Derived'].keys())
            # print(f['Derived/Particles_Average_Px'].keys(), species.value, f'Dervied/Particles_Average_Px/{species.value}')
            # vx = np.mean(f[f'Derived/Particles_Average_Px/{species.value}'], axis=(1,2)) / info[species].si_mass
            # vy = np.mean(f[f'Derived/Particles_Average_Py/{species.value}'], axis=(1,2)) / info[species].si_mass
            # T_3D = np.mean(f['Derived/temperature'], axis=(1,2))
            avg_E = np.mean(f[f'Derived/Average_Particle_Energy/{species.value}'], axis=(1,2))
        # E_kin = 0.5 * info[species].si_mass * (vx ** 2 + vy ** 2) / constants.electron_volt
        plt.plot(time * info.omega_pp, avg_E / avg_E[0], label=filename.stem[-3:]) # species.value
        # plt.plot(E_kin)
    # plt.yscale("log")
plt.legend()
plt.show()
    # break

In [ ]:
files = sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5"))
arr_k = np.empty(len(files))
arr_k_err = np.empty(len(files))
arr_omega = np.empty(len(files))
flow_velocity = np.empty(len(files))
for file_idx, filename in enumerate(files):
    flow_velocity[file_idx] = int(filename.stem[-3:])
    with h5py.File(filename) as f:
        x = f["Grid/grid/X"][:] / info.lambda_D
        y = f["Grid/grid/Y"][:] / info.lambda_D
        E_x = f['Electric Field/ex'][1:]
        E_y = f['Electric Field/ey'][1:]
        time = f["Header/time"][1:] * info.omega_pp
    res = analysis.fitGrowthRate(time, np.mean(E_x ** 2 + E_y ** 2, axis=(1,2)))
    linear_idx = slice(res[1][-1])
    E_field = E_x[linear_idx]
    # k-vector
    k, k_err = analysis.waveVector2D(x, y, E_field)
    arr_k[file_idx] = np.linalg.norm(k)
    arr_k_err[file_idx] = np.linalg.norm(k * k_err / np.linalg.norm(k))
    # frequency
    f, p = signal.periodogram(E_field, axis=-3, fs=1/(time[1] - time[0]))
    p_mean = np.mean(p, axis=(1,2))
    f *= 2 * np.pi
    def lorentzian( x, x0, gam, a, b):
        return a * gam**2 / ( gam**2 + ( x - x0 )**2) + b

    popt, pcov = optimize.curve_fit(
        lambda x, x0, gam, a, b: np.log(lorentzian(x, x0, gam, a, b)),
        f[f<np.pi][1:], np.log(p_mean[f<np.pi][1:]),
        p0=[f[1:][np.argmax(p_mean[1:])], 0.05, np.max(p_mean[1:]), 0])
    arr_omega[file_idx] = popt[0]

plt.errorbar(flow_velocity[:-1], (arr_omega / arr_k)[:-1], yerr=(arr_k_err / arr_omega)[:-1], ls="", marker="o")
with h5py.File(THEORY_U_ALPHA_FILE) as f:
    u_alpha = f["u_alpha_bulk"][:] * 1e-3
    gamma = f["gamma_max"][:] / info.omega_pp
    theta = f["theta_max"][:] * 180 / np.pi
    k_vec = f["k_max"][:] * info.lambda_D
    omega = f["omega_max"][:] / info.omega_pp

plt.plot(u_alpha, omega/k_vec)
plt.xlabel("Flow speed (km/s)")
plt.ylabel("Phase speed (1)")

In [ ]:
plots_2D.psdOmegaForAlphaFlowSpeed(info, "x", save=False)
plt.show()

In [ ]:
for species in [Species.ELECTRON]:
    plots_2D.psdFlowVelocity(info, species, "x", "x")
    plt.show()

In [ ]:
for species in Species:
    plots_2D.temperature3DOverTimeForAlphaFlowSpeed(
        info, species
    )

In [ ]:
files = sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5"))
n_points: int=10
species = Species.ELECTRON

plt.style.use(MPLSTYLE_FILE)
fig, axes = plt.subplots(1, 2, figsize=(FIGURE_FULL_SIZE[0] - 0.3, 3), sharex=True)
for ax, normalize_temperature in zip(axes, [False, True]):
    velocity = np.empty(len(files))
    T_diff = np.empty(len(files))
    T_diff_err = np.empty(len(files))
    for file_idx, filename in enumerate(files):
        velocity[file_idx] = int(filename.stem[-3:])
        with h5py.File(filename) as f:
            temp = physics.kelvinToElectronVolt(
                np.mean(f[f"Derived/Temperature/{species.value}"], axis=(1,2))
            )
        T_diff[file_idx] = np.mean(temp[-n_points:]) - np.mean(temp[:n_points])
        T_diff_err[file_idx] = np.sqrt(
            np.var(temp[-n_points:]) + np.var(temp[:n_points])
        ) / np.sqrt(n_points)
    if normalize_temperature:
        K_alpha_t0 = (info.alpha.si_mass * (velocity * 1e3) ** 2 / (2 * constants.electron_volt))
        T_diff /= K_alpha_t0
        T_diff_err /= K_alpha_t0
    else:
        T_diff /= info[species].temperature
        T_diff_err /= info[species].temperature
    ax.errorbar(
        velocity, T_diff, yerr=T_diff_err,
        marker="p" if normalize_temperature else "o",
        color="cornflowerblue" if normalize_temperature else "white",
        ls="", markeredgecolor="black", markeredgewidth=1
    )
    ax.set(
        xlim=(95, 185),
        xticks=np.arange(100, 185, 20),
    )
    if normalize_temperature:
        ax.ticklabel_format(style='sci', axis='y', scilimits=(-2,2), useMathText=True)
        ax.set_ylabel(f"Heating $\\Delta T_{species.symbol()}\\,/\\,K_\\alpha^{{t=0}}$ (1)")
    else:
        ax.set_ylabel(f"Heating $\\Delta T_{species.symbol()}\\,/\\,T_{species.symbol()}$ (eV)")
axes[0].set_xlabel(f"Flow velocity $u_\\alpha^{{t=0}}$ (km$\\,/\\,$s)")
axes[1].set_xlabel(f"Flow velocity $u_\\alpha^{{t=0}}$ (km$\\,/\\,$s)")
fig.tight_layout(h_pad=0.2)
plt.savefig(f"figures/svg/simulation-2D/alpha_flow_velocity_variation/heating_{species}.svg", bbox_inches="tight")

In [ ]:
plots_2D.heatingVsAlphaFlowVelocity(info, save=False)
plt.show()

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plots_2D.electricField2DSnapshot(filename, info, time=50.0, save=True)
    plt.show()
    break

In [ ]:
for filename in sorted((FOLDER_2D / "v_alpha_bulk_variation_By").glob("*.h5")):
    plots_2D.electricField2DSnapshot(filename, info, time=50.0)
    plt.show()

In [ ]:
for filename in sorted((FOLDER_2D / "v_alpha_bulk_variation_Bx_By").glob("*.h5")):
    plots_2D.energyEFieldOverTime(filename, info)
    plt.show()

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    # plots_2D.strengthBFieldOverTime(filename, info)
    plots_2D.psdBField(filename, info, True)
    plots_2D.energyBField(filename, info, save=True)
    plt.show()
    break

In [ ]:
for species in Species:
    plots_2D.flowVelocityVsTime(info, species, "x", "x")
    plt.show()

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plots_2D.energiesOverTime(filename, info)
    plt.show()
    break

In [ ]:
# Kinetic energies in the system
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plt.figure()
    K_species = []
    for species in Species:
        u_alpha = int(filename.stem[-3:])
        with h5py.File(filename) as f:
            time = f["Header/time"][:] * info.omega_pp
            x_grid = f[f"Grid/grid/X"][:]
            y_grid = f[f"Grid/grid/Y"][:]
            px_grid = f[f"Grid/px_py/{species.value}/Px"][:]
            py_grid = f[f"Grid/px_py/{species.value}/Py"][:]
            px_py = f[f'dist_fn/px_py/{species.value}'][:]
        # TODO: Wrong
        u_x, u_y = analysis.flowVelocity2D(x_grid, y_grid, px_grid, py_grid, px_py, info[species])
        K = 0.5 * info[species].si_mass * (u_x ** 2 + u_y ** 2) / constants.electron_volt
        K_species.append(info[species].number_density * K)
    for K_s, species in zip(K_species, Species):
        plt.plot(time, K_s * 1e-6, label=f"$W_{{{species.symbol()}}}$")
    # plt.plot(time[1:], W_total[1:] * 1e-6, label="$W_\\text{total}$")
    plt.yscale("log")
    plt.xlabel("Time $t\\,\\omega_{pp}$ (1)")
    plt.ylabel("Kinetic energy density K (MeV$\\,/\\,$m$^3$)")
    plt.xlim(0.0, 150.0)
    plt.legend(ncols=3, labelspacing=.2, columnspacing=0.5)
    plt.show()
    break

In [ ]:
species = Species.PROTON
files = sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5"))
labels = [int(p.stem[-3:]) for p in files]
general.momentumDistributionComparison(
    info, species, Distribution.X_PX, legend_ncols=2,
    files=files, labels=labels, times=150.0, legend_title=f"Flow velocity $u_\\alpha$ (km/s)")
plt.show()

In [ ]:
times = [0.0, 150.0]
species = Species.ALPHA
for i, filename in enumerate(sorted((FOLDER_2D / "v_alpha_bulk_variation").glob("*.h5"))):
    if i < 3: continue
    print(filename.stem[-3:])
    general.momentumDistributionComparison(info, species, Distribution.X_PX, filename, times, save=True)
    break
plt.show()

In [ ]:
species = Species.ALPHA
dist = Distribution.X_PX
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    general.spaceMomentumDistributon(info, species, dist, filename, time=50.0)
    plt.show()
    break

In [ ]:
species = Species.ALPHA
time = 60.0
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5"))[3:]:
    print(filename.stem[-3:])
    plots_2D.pxPyDistribution(info, species, filename, time)
    plt.show()
    break

In [ ]:
species = Species.PROTON
time = 150.0
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5"))[3:]:
    print(filename.stem[-3:])
    plots_2D.pxPyDistribution(info, species, filename, time)
    plt.show()
    break

In [ ]:
plt.rcParams["figure.dpi"] = 200
fig = plt.figure(constrained_layout=True, figsize=(5,4))
axes = fig.subplot_mosaic([["v_p", "v_a"],["f_p", "f_p"]], height_ratios=[1.5,1])
# fig, ax = plt.subplots(1, 2, figsize=(5,5))
filename = sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5"))[3]
plots_2D._pxPyDistSubplot(fig, axes["v_p"], info, filename, Species.PROTON, 54.4, (-3.5,7.17), (-5,5), False, True)
axes["v_p"].set_axis_off()
axes["v_p"].set_aspect("equal")
plots_2D._pxPyDistSubplot(fig, axes["v_a"], info, filename, Species.ALPHA, 54.4, (1.83, 7.6 * (3.5+ 7.17) / 10+1.7+11/(2 ** 9)), (-3.8,3.8), False, True)
axes["v_a"].set_axis_off()
axes["v_a"].set_aspect("equal")

filename = PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5"
v, f_v = general._loadSpaceMomDistribution(
    info, species, filename, Distribution.X_PX, 54.4, True
)
dv = abs(v[1] - v[0])
v = np.concat([[v[0]-dv], v]) + dv / 2

with h5py.File(filename) as f:
    if f"Grid/grid/X" in f:
        x_grid = f[f"Grid/grid/Px"][:] / info.lambda_D
    else:
        x_grid = f["Grid/grid"][:] / info.lambda_D

f_v[f_v == 0] = np.min(f_v[f_v>0])
axes["f_p"].pcolormesh(x_grid, v, f_v.T, norm="log", cmap=plt.colormaps["viridis"], rasterized=True)
# v_lim = (np.min(v), np.max(v))
# print(v_lim)
# axes["f_p"].set_ylim(-1.5, 8.5)
axes["f_p"].set_axis_off()

# plt.tight_layout()
plt.savefig("title.svg", bbox_inches="tight")

In [ ]:
filename = sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5"))[3]
plots_2D.velocitySpaceVsFlowVelocity(info, filename, save=True)
plt.show()

In [ ]:
plt.style.use(MPLSTYLE_FILE)
for time, regime in zip([55.0, 150.0], [True, False]):
    for species, xlim, ylim in zip([Species.PROTON, Species.ALPHA], [(-3, 9), (0, 9)], [(-6, 6), (-5, 5)]):
        fig, ax = plt.subplots(
            1, 1, sharex="col", layout="constrained",
            figsize=(2.52 if species == Species.PROTON else 2.5, FIGURE_HALF_SIZE[1]),
        )

        i = 0
        plots_2D._pxPyDistSubplot(fig, ax, info, filename, species, time, xlim, ylim, True, regime)
        ax.set_xlabel(f"Velocity $v_{{{species.symbol()},x}}\\,/\\,v^{{t=0}}_{{\\text{{t}}{species.symbol()}}}$ (1)")
        plt.savefig(f"figures/svg/simulation-2D/alpha_flow_velocity_variation/velocity_space-u_alpha=140-{species}_t={50 if regime else 150}.svg", bbox_inches="tight", transparent=True)
plt.show()

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    label = f"u_alpha={filename.stem[-3:]}"
    plots_2D.videoEFieldOverTime(info, filename, "x", label=label, save=True)

In [ ]:
filenames = sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5"))
species = Species.PROTON
normalized_velocity = True
time_steps = range(0, 1500, 20)
labels = [int(p.stem[-3:]) for p in filenames]
legend_title=f"Flow velocity $u_\\alpha$ (km/s)"
general.videoMomentumDistribution(
    info, Distribution.X_PX, species, filenames,
    save=False, labels=labels, legend_title=legend_title,
    legend_ncols=2)

In [ ]:
45e-9 / np.sqrt(constants.mu_0 * np.sum([i.si_mass * i.number_density for i in info]))

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    v = f"u_alpha={filename.stem[-3:]}"
    plots_2D.videoPxPyDistribution(info, species=Species.PROTON, filename=filename, save=True, label=v)

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    v = f"u_alpha={filename.stem[-3:]}"
    plots_2D.videoPxPyDistribution(info, species=Species.ALPHA, filename=filename, save=True, label=v)

## Magnetic Fields

In [ ]:
plots_2D.waveNumberVsMagneticField(info, save=True)
plots_2D.frequencyVsMagneticField(info, save=True)
plt.show()

In [ ]:
B = 45e-9
species = Species.ELECTRON
omega_c = np.abs(info[species].si_charge) * B / info[species].si_mass
omega_c / info.omega_pp * 150 * 2 * 180
print(omega_c / info.omega_pp)

In [ ]:
30e-9 / np.sqrt(constants.mu_0 * np.sum([i.si_mass * i.number_density for i in info]))

In [ ]:
alpha = np.linspace(0, 60 * np.pi / 180)
plt.plot(np.cos(alpha), np.sin(alpha))
plt.gca().set_axis_off()
plt.gca().set_aspect("equal")
plt.savefig("figures/svg/simulation-2D/magnetic_fields/magnetic_field_proton_trajectory.svg", bbox_inches="tight", transparent=True)
plt.show()

In [ ]:
for species in Species:
    plots_2D.heatingVsMagneticField(info, species, save=True)
    plt.show()

In [ ]:
# plot 3D temperature vs time for different magnetic fields.
species = Species.PROTON
plt.style.use(MPLSTYLE_FILE)
plt.figure()

leg_u100 = []
leg_u140 = []
for u_label in [100, 140]:
    for folder_id, label in zip(["", "_Bx", "_By", "_Bz", "_Bx_By"], ["B=0", "B_x>0", "B_y>0", "B_z>0", "B_x=B_y"]):
        for filename in sorted((FOLDER_2D / f"v_alpha_bulk_variation{folder_id}").glob("*.h5")):
            u = int(filename.stem[-3:])
            if u != u_label:
                continue
            with h5py.File(filename) as f:
                time = f["Header/time"][:] * info.omega_pp
                temp = np.mean(f[f"Derived/Temperature/{species.value}"], axis=(1,2))
            line = plt.plot(
                time, physics.kelvinToElectronVolt(temp),
                # label=f"(${label}$) $u_\\alpha^{{t=0}}$ = {v}"
                label=f"${label}$"
            )[0]
            (leg_u100 if u_label == 100 else leg_u140).append(line)
leg1 = plt.legend(handles=leg_u100, loc="upper left", title="$u_\\alpha^{t=0}$=100 km/s",ncols=1 if species == Species.ELECTRON else 2, columnspacing=0.5, labelspacing=0.5, fancybox=False, framealpha=0.4)
plt.legend(handles=leg_u140, title="$u_\\alpha^{t=0}$=140 km/s", loc="lower right", ncols=3 if species == Species.ELECTRON else 2, columnspacing=0.5, labelspacing=0.5, fancybox=False, framealpha=0.4)
plt.gca().add_artist(leg1)
plt.xlabel("Time $t\\,\\omega_{pp}$ (1)")
plt.ylabel(f"Temperature $T_{species.symbol()}$ (eV)")
plt.xlim(0, 150.0)
y_low = 1.5 if species == Species.PROTON else 5
plt.ylim(bottom=2 if species == Species.PROTON else 10 if species == Species.ALPHA else 99.7)
plt.tight_layout(pad=0.2)
# plt.savefig(f"figures/svg/simulation-2D/magnetic-field-directions/temperature_{species}-vs-time.svg", bbox_inches="tight")
plt.show()

# Simulation 1D

## Density variation

In [ ]:
for filename in sorted(DENSITY_VARIATION_FOLDER.glob("*.h5")):
    plots_1D.electricFieldDensityRatio(filename, True)
    break

In [ ]:
plots_1D.growthRateDensityRatio(save=True)
plots_1D.linearRegimeDensityRatio(save=True)
plt.show()

In [ ]:
plots_1D.frequencyVsDensityRatio(save=False)
plots_1D.wavenumberVsDensityRatio(save=False)
plt.show()

In [ ]:
for species in Species:
    plots_1D.heatingVsDensityRatio(species, save=True)
plt.show()

In [ ]:
plots_1D.energyFractionsDensityRatio(save=True)

In [ ]:
for filename in sorted(DENSITY_VARIATION_FOLDER.glob("density_*.h5")):
    print(filename)
    ratio = 10 ** float(filename.stem.split("_")[-1])
    info = plots_1D.runInfoForDenistyRatio(ratio)
    plots_1D.energyEFieldOverTime(
        filename=filename, info=info
    )
    plt.show()

## Evolution of Simulation (1D - 8192) 

In [ ]:
for species, vlim, vticks in zip([Species.PROTON, Species.ALPHA],[(-3, 8), (-1, 7)], [[0,4,8], [0,2,4,6]]):
    dist_type = Distribution.X_PX
    filename = PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5"
    time = range(0, 1500, 5)
    general.videoEvolutionDistributionFunction(
        info, species, filename, dist_type, time,
        vlim=vlim, vticks=vticks, save=True
    )
plt.show()

In [ ]:
times = [0.0, 50.0, 150.0]
for species, y_bot, x_lim, loc in zip(
    Species,
    [1e-4, 2e-1, 1e-1],
    [(-4, 4), (-4, 10), (-1, 8)],
    [(0.22, 0.05), (0.42, 0.67), (0.3, 0.04)]
):
    # if species == Species.ELECTRON: continue
    general.momentumDistributionComparison(
        info, species, Distribution.X_PX,
        PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5",
        times, legend_loc=loc, legend_title="",
        labels=[f"$t\\,\\omega_\\text{{pp}}={int(t)}$" for t in times],
        y_lim=(y_bot, 1e3 if species == Species.PROTON else None),
        x_lim=x_lim,
        x_ticks=[-4,0,4,8] if species == Species.PROTON else None,
        save=True,
        save_folder="simulation-1D",
    )
plt.show()

In [ ]:
for idx, (species, v_lim, v_ticks) in enumerate(zip(
    [Species.PROTON, Species.ALPHA],
    [(-4, 10), (-1, 8)],
    [np.linspace(-4, 8, 4), np.linspace(0, 8, 3)],
)):
    general.spaceVelocityDistributionMulti(
        info, species, Distribution.X_PX,
        PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5",
        times=[0.0, 60.0, 150.0],
        v_lim=v_lim,
        v_ticks=v_ticks,
        subfig_offset=idx * 3,
        save=True,
        save_folder="simulation-1D"
    )

In [ ]:
plots_1D.eFieldEvolutionCombined(
    filename=PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5",
    info=info, save=True
)

In [ ]:
plots_1D.electricFieldOverSpaceAndTime(
    filename=PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5",
    info=info, save=True
)
plt.show()

In [ ]:
plots_1D.energyEFieldOverTime(
    filename=PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5",
    info=info, save=True, show_fit_details=False
)
plt.show()

In [ ]:
species = Species.ELECTRON
with h5py.File(PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5") as f:
    time = f["Header/time"][:]
    dist = np.mean(f[f"/dist_fn/x_px/{species.value}"], axis=1)
    x_grid = f[f"Grid/x_px/{species.value}/X"][:]
    px_grid = f[f"Grid/x_px/{species.value}/Px"][:]

def mean_h(val, freq):
    return np.sum(val * freq, axis=1) / np.sum(freq, axis=1)

def var_h(val, freq):
    dev = freq * (val - mean_h(val, freq)[:,None]) ** 2
    return np.sum(dev, axis=1) / np.sum(freq, axis=1)
mean = mean_h(px_grid, dist)
var = var_h(px_grid, dist)
plt.plot(mean / info[species].si_mass / info[species].v_thermal)
print((mean[-1] - mean[0])/ info[species].si_mass / info[species].v_thermal)
# cdf = np.cumsum(dist, axis=1) / np.sum(dist, axis=1)[:,None]
# from scipy import special
# cdf_norm = 0.5 * (1 + special.erf((px_grid - mean[:,None]) / np.sqrt(2 * var[:,None])))

# pdf = dist / np.sum(dist * (px_grid[:,1] - px_grid[:,0])[:,None], axis=1)[:,None]
# pdf_norm = np.exp(- (px_grid - mean[:,None]) ** 2 / (2 * var[:,None])) / np.sqrt(2 * np.pi * var[:,None])
# pdf_norm /= np.sum(pdf_norm * (px_grid[:,1] - px_grid[:,0])[:,None], axis=1)[:,None]
# plt.plot(pdf_norm[0])
# plt.plot(pdf[0])
# res = stats.chisquare(pdf, pdf_norm, ddof = 2, axis=1)
# print(res)
# Dn = np.max(np.abs(cdf - cdf_norm), axis=1)
# Kn = np.sqrt(cdf.shape[1]) * Dn
# plt.plot(special.kolmogorov(Kn))
# plt.plot(stats.distributions.ksone.sf(Dn, cdf.shape[1]))
# plt.plot(Dn)
# plt.yscale("log")

In [ ]:
for species in Species:
    plots_1D.avgTemperatureXOverTime(
        filename=PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5",
        info=info, species=species, save=True
    )
plt.show()

In [ ]:
plots_1D.velocityDistributionOverTimeCombined(
    PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5",
    info,
    times=[0.0, 150.0],
    v_lim_arr=[(-4, 4), (-4, 10), (-1, 8)],
    v_tick_arr=[[-3,0,3], [-3,0,3,6,9], [0,4,8]],
    save=False,
)

In [ ]:
plt.rcParams["figure.dpi"] = 200
plots_1D.energyEFieldOverTime(
    filename=V_FLOW_VARIATION_FOLDER / "v_alpha_bulk_100.h5",
    info=info, save=False
)
plt.show()

In [ ]:
plots_1D.linearTheoryWaveProperties(info, save=True)

In [ ]:
print(np.sqrt(2 * np.abs(info.proton.si_charge) * 1 * (2 * np.pi * info.lambda_D / 0.7) / info.proton.si_mass) / info.proton.v_thermal)

In [ ]:
u_alpha = 100_000
v_phase = u_alpha - np.sqrt(3/2) * info.alpha.v_thermal
df_alpha = - 2 * ((v_phase - u_alpha) / info.alpha.v_thermal ** 2) * (info.alpha.v_thermal / np.pi) ** (3/2) * np.exp(- ((v_phase - u_alpha) / info.alpha.v_thermal) ** 2)
df_elec = - 10 * 2 * (v_phase / info.electron.v_thermal ** 2) * (info.electron.v_thermal / np.pi) ** (3/2) * np.exp(- (v_phase / info.electron.v_thermal) ** 2)
df_prot = - 2 * 2 * (v_phase / info.proton.v_thermal ** 2) * (info.proton.v_thermal / np.pi) ** (3/2) * np.exp(- (v_phase / info.proton.v_thermal) ** 2)
print(df_elec / df_alpha, df_prot / df_alpha)

In [ ]:
plots_1D.illustrateSimulationGrid(save=True)
plots_1D.illustrateVelocitySpace(info, u_alpha=None, save=True)
plots_1D.illustrateVelocitySpace(info, u_alpha=120, save=True)
plt.show()

## Particle variation

In [ ]:
plots_1D.particleVariationEnergyVsTime(info=info, save=False)
plt.show()

In [ ]:
for species in Species:
    plots_1D.particleVariationTemperature3D(species, save=save)
plt.show()

In [ ]:
for species in Species:
    plots_1D.particleVariationTemperatureXDiff(info, species, save=True)
plt.show()

In [ ]:
for species in Species:
    plots_1D.particleVariationTemperatureXVsTime(info, species, save=save)
plt.show()

In [ ]:
plots_1D.particleVariationGrowthRate(info, save=True)
plt.show()

In [ ]:
plots_1D.particleVariationWavenumber(info=info, save=True)
plots_1D.particleVariationFrequency(info=info, save=True)
plt.show()

In [ ]:
with h5py.File(THEORY_U_ALPHA_FILE) as f:
    print(f.keys())
    print(np.diff(f['search_space/theta']))
    u_alpha = f['u_alpha_bulk'][:]
    idx = np.argmin(np.abs(u_alpha - 100_000))
    k_max = f['k_max'][idx] * info.lambda_D
    theta_max = f['theta_max'][idx]
    print(k_max * (np.cos(theta_max - 0.00349066) - np.cos(theta_max + 0.00349066 )))